In [1]:
import sys
sys.path.insert(0, '/home/bettyld/PJ/Documents/NLP_PJ/nlp_recommend')

In [7]:
from nlp_recommend.utils.load_data import LoadData
corpus = LoadData(n_max=5000, random=True)
philo_df = corpus.corpus
philo_df.head()

Removing numbers...
Removing proper noun... 4837


Processing: 100%|█████████████████████████████████| 6/6 [00:13<00:00,  2.21s/it]


Cleaning sentences...
Index(['index', 'sentence', 'author', 'title', 'clean_sentence',
       'tok_lem_sentence'],
      dtype='object')


,level_0,index,sentence,author,title,clean_sentence,tok_lem_sentence
0,2513,76899,"And of the art of instruction, shall we say th...",Plato,Phaedrus,and of the art of instruction shall we say tha...,"[art, instruction, shall, say, one, many, kind]"
1,2514,133124,Their appearance presence change and disappear...,Dasgupta,"A History of Indian Philosophy, Volume 1",their appearance presence change and disappear...,"[appearance, presence, change, disappearance, ..."
2,2516,46612,It was not long before the effects of these al...,Hume,"The history of england, under the house of tudor.",it was not long before the effects of these al...,"[long, effect, alteration, seen]"
3,2518,133276,"In this state there is nothing which is known,...",Dasgupta,"A History of Indian Philosophy, Volume 1",in this state there is nothing which is known ...,"[state, nothing, known, even, knowledge, pheno..."
4,2522,129669,They might be opposed as philosophy and rhetor...,Plato,Theaetetus,they might be opposed as philosophy and rhetor...,"[might, opposed, philosophy, rhetoric, convers..."


# SBert

In [8]:
from sentence_transformers import SentenceTransformer, util
import torch


model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [9]:
#Compute embeddings
corpus_embeddings = model.encode(philo_df.sentence.values, convert_to_tensor=True)


In [10]:
#Query sentences
query_sentence = 'I am hungry, where is my dinner?'

In [27]:
import pandas as pd
pd.options.display.max_colwidth = 500

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus_embeddings))

query_embedding = model.encode(query_sentence, convert_to_tensor=True)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)
# top_results = top_results.cpu().data.numpy().argmax()
print("\n\n======================\n\n")
print("Query:", query_sentence)
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    score = score.cpu().data.numpy() 
    idx = idx.cpu().data.numpy()
    print(philo_df[['sentence', 'author']].iloc[idx], "(Score: {:.4f})".format(score))





Query: I am hungry, where is my dinner?

Top 5 most similar sentences in corpus:
sentence    I said painful, but that to eat when you are hungry is pleasant.
author                                                                 Plato
Name: 1237, dtype: object (Score: 0.5173)
sentence    And as to Wagnerian leitmotif, I fear I lack the necessary culinary understanding for it.
author                                                                                      Nietzsche
Name: 792, dtype: object (Score: 0.3368)
sentence    Hunger is no evidence that there is food at hand to appease it.
author                                                            Nietzsche
Name: 755, dtype: object (Score: 0.3335)
sentence    For that I am too weak.
author                    Nietzsche
Name: 312, dtype: object (Score: 0.2912)
sentence    It should be cooked for the female after parturition.
author                                                  Aristotle
Name: 182, dtype: object (Score: 0.2